# Silver Layer Transformation - Telemetry Data

## Objective
Transform the `bronze.telemetry_data` table into a clean Silver table for detailed car performance analysis.

## Steps
1. **Read** data from Bronze table `bronze.telemetry_data`.
2. **Select & Rename** important columns:
   - Driver info: `Driver`
   - Car telemetry: `nGear`, `Throttle`, `Brake`, `DRS`, `RPM`, `Speed`, `Distance`
   - Time-related: `Time`, `SessionTime`, `LapNumber`, `Date`
   - Metadata: `Source`
3. **Data Cleaning**
   - Cast numeric fields (`nGear`, `Throttle`, `Brake`, `DRS`, `RPM`, `Speed`, `Distance`, `LapNumber`) into proper numeric types.
   - Convert `Date` to `timestamp`.
   - Trim text fields (`Driver`, `Source`).
   - Drop duplicates and null-heavy rows.
4. **Write** results into Silver schema as `silver.telemetry_data`.

## Output
- Clean Silver table:
  `silver.telemetry_data`


In [0]:
from pyspark.sql import functions as F

# Read from Bronze
telemetry_bronze = spark.table("bronze.telemetry_data")

# Transformations
telemetry_silver = (
    telemetry_bronze
    .select(
        F.trim(F.col("Driver")).alias("driver"),
        F.col("LapNumber").cast("int").alias("lap_number"),
        F.col("nGear").cast("int").alias("gear"),
        F.col("Throttle").cast("float").alias("throttle"),
        F.col("Brake").cast("float").alias("brake"),
        F.col("DRS").cast("int").alias("drs"),
        F.col("RPM").cast("int").alias("rpm"),
        F.col("Speed").cast("float").alias("speed"),
        F.col("Distance").cast("float").alias("distance"),
        F.col("SessionTime").cast("double").alias("session_time_sec"),
        F.col("Time").cast("double").alias("relative_time_sec"),
        F.to_timestamp("Date").alias("event_timestamp"),
        F.trim(F.col("Source")).alias("source")
    )
    .dropDuplicates()
)

# Write to Silver
(
    telemetry_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver.telemetry_data")
)